In [1]:
import tempfile

In [2]:
# El siguiente ejemplo muestra cómo crear un archivo temporal con la función TemporaryFile(), en modo binario:

# Crea un archivo temporal en modo binario
temporal1 = tempfile.TemporaryFile()

# Muestra información del objeto creado
print(temporal1)  # _io .bufferedrandom="" name="4"
print(type(temporal1))  # class io.bufferedrandom=""
print('nombre :',temporal1.name)  # identificador del archivo, 4

# Escribe en el fichero temporal
temporal1.write(b'archivos temporales\ncon Python')

# Sitúa el puntero al comienzo del archivo
temporal1.seek(0)

# Lee archivo temporal desde su comienzo
lectura1 = temporal1.read()

# Muestra la información leída
print(lectura1)

# Cierra y elimina el archivo temporal 
temporal1.close()

<_io.BufferedRandom name=42>
<class '_io.BufferedRandom'>
42
b'archivos temporales\ncon Python'


In [4]:
# En el ejemplo que sigue se muestra cómo crear un archivo temporal en modo texto, con la posibilidad de capturar los errores que pudieran producirse al acceder al sistema de archivos:

import tempfile

# Crea un archivo temporal en modo texto
temporal2 = tempfile.TemporaryFile(mode='w+t')
print('nombre: ',temporal2.name)  # identificador del archivo, 4

# Captura posibles errores de acceso al sistema de archvios
try:
    # Escribe tres líneas en el archivo temporal   
    temporal2.writelines(['Linea 1\n', 'Linea 2\n', 'Linea 3\n'])  

    # Sitúa el puntero al comienzo del archivo
    temporal2.seek(0)
    
    # Lee y muestra todas las líneas del archivo temporal
    for linea in temporal2:
        print(linea.rstrip())

finally:
    # Cierra y elimina el archivo temporal
    temporal2.close()

nombre:  42
Linea 1
Linea 2
Linea 3
